In [288]:
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import numpy as np
import random
import math

**Таблица сопряжённости** (матрица неточности) содержит сводные показатели качества работы классификатора.
* **Строки** соответствуют **фактическим** классам тестового набора;
* **Cтолбцы** соответствуют **предсказанным** классом.

Таблица содержит четыре сводных показателя, каждый из которых отражает количество объектов в одной и четырех
категорий:
* **Истинно позитивный** (*True positive*, **TP**) -- объект
класса `1` был верно помечен меткой `1`;
* **Ложно позитивный** (*False positive*, **FP**) -- объект
фактически принадлежит классу `0`, но помечен меткой `1`;
* **Истинно отрицательный** (*True negative*, **TN**) -- классификатор
верно определил, что объект класса `0` принадлежит классу `0`;
* **Ложно отрицательный** (*False negative*, **FN**) -- классификатор
пометил объект меткой `0`, однако на самом деле объект принадлежит классу `1`.


|                    | Предсказано `0` | Предсказано `1` |
|:-------------------|:----------------|:----------------|
| **Фактически** `0` | TN              | FP              |
| **Фактически** `1` | FN              | TP              |


In [289]:
class ConfusionMatrix:
    def __init__(self, y_true=None, y_pred=None):
        self._confusion_matrix = np.array([[0,0], [0,0]])
        if not (y_true is None or y_pred is None):
            self.upd(y_true, y_pred)

    def upd(self, y_true, y_pred):
        for value, prediction in zip(y_true, y_pred):
            self._confusion_matrix[value][prediction] += 1

    def get(self):
        return self._confusion_matrix

    def get_flatten(self):
        return self._confusion_matrix.ravel()

In [290]:
def f1_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return tp / (tp + (fp + fn) / 2)

def accuracy_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return (tn + tp) / (tn + fp + fn + tp)

def precision_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return tp / (tp + fp)

def recall_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return tp / (fn + tp)

In [291]:
def roc_curve(y_true, y_proba, partitions=100):
    tpr, fpr = [], []
    thresholds = np.linspace(0, 1, partitions)

    for threshold in thresholds:
        y_pred = np.where(y_proba >= threshold, 1, 0)
        tn, fp, fn, tp = ConfusionMatrix(y_true, y_pred).get_flatten()
        fpr.append(fp / (fp + tn))
        tpr.append(tp / (tp + fn))
    return fpr, tpr, thresholds

def auc(x, y):
    s = 0
    for i in range(1, len(x)):
        s += 0.5 * (x[i] - x[i - 1]) * (y[i] + y[i - 1])
    return s